In [2]:
%matplotlib inline 
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/',one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [34]:
data.test.cls = np.argmax(data.test.labels ,axis =1)

In [4]:
img_size = 28
img_size_flat = img_size*img_size
img_shape= (img_size,img_size)
num_channels = 1
num_classes = 10

# MODEL

## TF GRAPH INPUTS

In [5]:
x = tf.placeholder(tf.float32,shape=[None,img_size_flat],name='x')
x_image = tf.reshape(x,[-1,img_size,img_size,num_channels]) 

y_true = tf.placeholder(tf.float32,shape=[None,num_classes],name='y_true')
y_true_cls = tf.arg_max(y_true,dimension=1)

## TF LAYERS

In [6]:
net = x_image
net = tf.layers.conv2d(inputs=net,name='layer_conv1',padding='same',
                       filters=16,kernel_size=5,activation=tf.nn.relu)

In [7]:
layer_conv1 = net

In [9]:
net = tf.layers.max_pooling2d(inputs=net ,pool_size=2,strides=2)

In [10]:
net = tf.layers.conv2d(inputs=net,name='layer_conv2',padding='same',
                       filters=36,kernel_size=5,activation=tf.nn.relu)

In [11]:
layer_conv2= net

In [12]:
net = tf.layers.max_pooling2d(inputs=net,pool_size=2,strides=2)

In [13]:
print layer_conv1
print layer_conv2
print net

Tensor("layer_conv1/Relu:0", shape=(?, 28, 28, 16), dtype=float32)
Tensor("layer_conv2/Relu:0", shape=(?, 14, 14, 36), dtype=float32)
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 7, 7, 36), dtype=float32)


In [14]:
net = tf.contrib.layers.flatten(net)
print net

Tensor("Flatten/Reshape:0", shape=(?, 1764), dtype=float32)


In [16]:
net = tf.layers.dense(inputs=net,name='layer_fc1',units=128,activation=tf.nn.relu)

In [17]:
net = tf.layers.dense(inputs=net,name='layer_fc_out',units=num_classes,activation=tf.nn.relu)

## Logits
Output of the final layer before softmax is called Logits.

In [18]:
logits = net

In [19]:
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.arg_max(y_pred,dimension=1)

## Loss & Optimizer 

In [21]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_true)

In [22]:
loss = tf.reduce_mean(cross_entropy)

In [23]:
optimizer = tf.train.AdamOptimizer?

In [24]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

In [25]:
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) :
    print var

Tensor("layer_conv1/kernel/read:0", shape=(5, 5, 1, 16), dtype=float32)
Tensor("layer_conv1/bias/read:0", shape=(16,), dtype=float32)
Tensor("layer_conv2/kernel/read:0", shape=(5, 5, 16, 36), dtype=float32)
Tensor("layer_conv2/bias/read:0", shape=(36,), dtype=float32)
Tensor("layer_fc1/kernel/read:0", shape=(1764, 128), dtype=float32)
Tensor("layer_fc1/bias/read:0", shape=(128,), dtype=float32)
Tensor("layer_fc_out/kernel/read:0", shape=(128, 10), dtype=float32)
Tensor("layer_fc_out/bias/read:0", shape=(10,), dtype=float32)
Tensor("beta1_power/read:0", shape=(), dtype=float32)
Tensor("beta2_power/read:0", shape=(), dtype=float32)
Tensor("layer_conv1/kernel/Adam/read:0", shape=(5, 5, 1, 16), dtype=float32)
Tensor("layer_conv1/kernel/Adam_1/read:0", shape=(5, 5, 1, 16), dtype=float32)
Tensor("layer_conv1/bias/Adam/read:0", shape=(16,), dtype=float32)
Tensor("layer_conv1/bias/Adam_1/read:0", shape=(16,), dtype=float32)
Tensor("layer_conv2/kernel/Adam/read:0", shape=(5, 5, 16, 36), dtype=f

In [27]:
def get_weight_variables(layername):
    with tf.variable_scope(layername,reuse=True) :
        variable = tf.get_variable('kernel')
    return variable

## TF SESSION RUN

In [29]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [30]:
train_batch_size = 128

In [31]:
total_iterations = 0
def optimize(num_iterations):
    global total_iterations  
    current_iter = 0
    try :
        for i in range(total_iterations,total_iterations+num_iterations):
            x_batch,y_true_batch = data.train.next_batch(train_batch_size)
            feed_dict = {x:x_batch,y_true:y_true_batch}
            session.run(optimizer,feed_dict=feed_dict)
            current_iter = i
        total_iterations += num_iterations
    except :
        total_iterations += current_iter

In [35]:
test_batch_size = 256
def print_accuracy(show_example_errors= False ,show_confusion_matrix = False) :
    num_test = len(data.test.images)
    cls_pred = np.zeros(shape=num_test,dtype = np.int)
    i=0
    while(i<num_test) :
        j=min(i + test_batch_size, num_test)
        images = data.test.images[i:j,:]
        labels = data.test.labels[i:j,:]
        feed_dict={x:images,y_true:labels}
        cls_pred[i:j] = session.run(y_pred_cls,feed_dict=feed_dict)
        i=j
    cls_true = data.test.cls
    correct = (cls_true==cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum)/num_test   
    print "Accuracy on Test-Set: {0:.1%} ({1} / {2})".format(acc, correct_sum, num_test)

# MODEL RUN

In [36]:
print_accuracy()

Accuracy on Test-Set: 10.6% (1057 / 10000)


In [37]:
optimize(1)
print_accuracy()

Accuracy on Test-Set: 11.5% (1147 / 10000)


In [38]:
optimize(999)
print_accuracy()

Accuracy on Test-Set: 95.4% (9536 / 10000)
